In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

/home/fairouz/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
dvf_test = pd.read_csv("./dvf_test.csv")

In [3]:
print(dvf_test.shape)

(1391, 48)


# Data preparation

In [4]:
dvf_test.head()

,index,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,...,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,lon,lat,code_IRIS,code_district_admin,code_district_custom
0,249349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1530,...,78.0,4.0,NaN,NaN,NaN,2.252551,48.837822,751166131.0,61.0,Auteuil Sud
1,249400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1532,...,34.0,1.0,NaN,NaN,NaN,2.282171,48.831289,751155732.0,57.0,Vaugirard-Parc des Expositions
2,249800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1528,...,34.0,2.0,NaN,NaN,NaN,2.301586,48.873205,751083005.0,30.0,Hoche-Friedland
3,249943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1528,...,59.0,2.0,NaN,NaN,NaN,2.364598,48.857983,751031102.0,11.0,Archives
4,250150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1529,...,20.0,1.0,NaN,NaN,NaN,2.365183,48.861299,751031002.0,10.0,Enfants Rouges


In [5]:
dvf_test.duplicated().sum()

0

In [6]:
dvf_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1391 entries, 0 to 1390
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       1391 non-null   int64  
 1   Code service CH             0 non-null      float64
 2   Reference document          0 non-null      float64
 3   1 Articles CGI              0 non-null      float64
 4   2 Articles CGI              0 non-null      float64
 5   3 Articles CGI              0 non-null      float64
 6   4 Articles CGI              0 non-null      float64
 7   5 Articles CGI              0 non-null      float64
 8   No disposition              1391 non-null   int64  
 9   Date mutation               1391 non-null   int64  
 10  Nature mutation             1391 non-null   object 
 11  No voie                     1391 non-null   float64
 12  B/T/Q                       59 non-null     object 
 13  Type de voie                1390 

In [7]:
# Taux des données manquantes
dvf_test.isnull().sum()/len(dvf_test)*100

index                           0.000000
Code service CH               100.000000
Reference document            100.000000
1 Articles CGI                100.000000
2 Articles CGI                100.000000
3 Articles CGI                100.000000
4 Articles CGI                100.000000
5 Articles CGI                100.000000
No disposition                  0.000000
Date mutation                   0.000000
Nature mutation                 0.000000
No voie                         0.000000
B/T/Q                          95.758447
Type de voie                    0.071891
Code voie                       0.000000
Voie                            0.000000
Code postal                     0.000000
Commune                         0.000000
Code departement                0.000000
Code commune                    0.000000
Prefixe de section            100.000000
Section                         0.000000
No plan                         0.000000
No Volume                     100.000000
1er lot         

In [8]:
#Ici, nous constatons que la base brute contient beaucoup de variables qui ont un taux de plus de 90% des données manquantes. Donc, pour ces variables, nous avons exclué dans notre jeu de données.
#Pour les autres variables qui ont un taux de moins de 40%, nous utilisons la méthode de l'imputation de données pour remplacer les données manquantes 
for col in dvf_test.columns:
    if dvf_test[col].isnull().sum()/len(dvf_test)*100 > 78:
        dvf_test.drop(columns = col, inplace = True)
        
dvf_test.shape

(1391, 27)

In [9]:
dvf_test.columns

Index(['index', 'No disposition', 'Date mutation', 'Nature mutation',
       'No voie', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Section', 'No plan',
       '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 'Nombre de lots',
       'Code type local', 'Type local', 'Surface reelle bati',
       'Nombre pieces principales', 'lon', 'lat', 'code_IRIS',
       'code_district_admin', 'code_district_custom'],
      dtype='object')

In [10]:
index = ['index']
date = ['Date mutation']
data_quali = ['Nature mutation', 'Type de voie', 'Voie', 'Code postal', 'Commune', 'Code departement', 'Type local']
data_num = ['1er lot', 'Surface Carrez du 1er lot', 'Nombre de lots','Surface reelle bati','Nombre pieces principales', 'lon', 'lat']

In [11]:
for col in dvf_test.columns:
    if col not in index + data_quali + data_num:
        dvf_test.drop(columns = col, inplace = True)
        
dvf_test.shape

(1391, 15)

In [12]:
dvf_test.head()

,index,Nature mutation,Type de voie,Voie,Code postal,Commune,Code departement,1er lot,Surface Carrez du 1er lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,lon,lat
0,249349,Vente,AV,FERDINAND BUISSON,75016,BOULOGNE-BILLANCOURT,92,9.0,"79,40",1,Appartement,78.0,4.0,2.252551,48.837822
1,249400,Vente,RUE,D ORADOUR SUR GLANE,75015,ISSY-LES-MOULINEAUX,92,368.0,"32,88",1,Appartement,34.0,1.0,2.282171,48.831289
2,249800,Vente,RUE,CHATEAUBRIAND,75008,PARIS 08,75,511.0,"33,66",1,Appartement,34.0,2.0,2.301586,48.873205
3,249943,Vente,RUE,DE TURENNE,75003,PARIS 03,75,7.0,"50,67",1,Appartement,59.0,2.0,2.364598,48.857983
4,250150,Vente,RUE,DU PONT AUX CHOUX,75003,PARIS 03,75,25.0,"19,70",1,Appartement,20.0,1.0,2.365183,48.861299


In [13]:
dvf_test.isna().sum()

index                          0
Nature mutation                0
Type de voie                   1
Voie                           0
Code postal                    0
Commune                        0
Code departement               0
1er lot                        2
Surface Carrez du 1er lot    483
Nombre de lots                 0
Type local                     0
Surface reelle bati            0
Nombre pieces principales      0
lon                            1
lat                            1
dtype: int64

In [14]:
dvf_test = dvf_test[dvf_test["Surface Carrez du 1er lot"].notna()]
dvf_test.dropna(inplace = True)

In [15]:
dvf_test.to_csv("./data_test_clean.csv")
dvf_test = pd.read_csv("./data_test_clean.csv")
dvf_test.head()

,Unnamed: 0,index,Nature mutation,Type de voie,Voie,Code postal,Commune,Code departement,1er lot,Surface Carrez du 1er lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,lon,lat
0,0,249349,Vente,AV,FERDINAND BUISSON,75016,BOULOGNE-BILLANCOURT,92,9.0,"79,40",1,Appartement,78.0,4.0,2.252551,48.837822
1,1,249400,Vente,RUE,D ORADOUR SUR GLANE,75015,ISSY-LES-MOULINEAUX,92,368.0,"32,88",1,Appartement,34.0,1.0,2.282171,48.831289
2,2,249800,Vente,RUE,CHATEAUBRIAND,75008,PARIS 08,75,511.0,"33,66",1,Appartement,34.0,2.0,2.301586,48.873205
3,3,249943,Vente,RUE,DE TURENNE,75003,PARIS 03,75,7.0,"50,67",1,Appartement,59.0,2.0,2.364598,48.857983
4,4,250150,Vente,RUE,DU PONT AUX CHOUX,75003,PARIS 03,75,25.0,"19,70",1,Appartement,20.0,1.0,2.365183,48.861299


In [16]:
dvf_test.drop(columns = "Unnamed: 0", inplace = True)
print(dvf_test.isna().sum())
print(dvf_test.shape)
dvf_test.head()

index                        0
Nature mutation              0
Type de voie                 0
Voie                         0
Code postal                  0
Commune                      0
Code departement             0
1er lot                      0
Surface Carrez du 1er lot    0
Nombre de lots               0
Type local                   0
Surface reelle bati          0
Nombre pieces principales    0
lon                          0
lat                          0
dtype: int64
(907, 15)


,index,Nature mutation,Type de voie,Voie,Code postal,Commune,Code departement,1er lot,Surface Carrez du 1er lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,lon,lat
0,249349,Vente,AV,FERDINAND BUISSON,75016,BOULOGNE-BILLANCOURT,92,9.0,"79,40",1,Appartement,78.0,4.0,2.252551,48.837822
1,249400,Vente,RUE,D ORADOUR SUR GLANE,75015,ISSY-LES-MOULINEAUX,92,368.0,"32,88",1,Appartement,34.0,1.0,2.282171,48.831289
2,249800,Vente,RUE,CHATEAUBRIAND,75008,PARIS 08,75,511.0,"33,66",1,Appartement,34.0,2.0,2.301586,48.873205
3,249943,Vente,RUE,DE TURENNE,75003,PARIS 03,75,7.0,"50,67",1,Appartement,59.0,2.0,2.364598,48.857983
4,250150,Vente,RUE,DU PONT AUX CHOUX,75003,PARIS 03,75,25.0,"19,70",1,Appartement,20.0,1.0,2.365183,48.861299


In [17]:
for i in range(len(dvf_test)):
    #dvf_train["Valeur fonciere"][i] = dvf_train["Valeur fonciere"][i].replace(',', '.')
    #dvf_test["Valeur fonciere"][i] = float(dvf_test["Valeur fonciere"][i].replace(',', '.'))
    dvf_test["Surface Carrez du 1er lot"][i] = float(dvf_test["Surface Carrez du 1er lot"][i].replace(',', '.'))

In [18]:
for col in data_quali:
    dvf_test[col] = dvf_test[col].astype('object')
for col in data_num:
    #dvf_test[col] = dvf_test[col].astype('float')
    dvf_test[col] = pd.to_numeric(dvf_test[col], errors = 'coerce')

In [19]:
print(dvf_test.shape)
dvf_test.head()

(907, 15)


,index,Nature mutation,Type de voie,Voie,Code postal,Commune,Code departement,1er lot,Surface Carrez du 1er lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,lon,lat
0,249349,Vente,AV,FERDINAND BUISSON,75016,BOULOGNE-BILLANCOURT,92,9.0,79.40,1,Appartement,78.0,4.0,2.252551,48.837822
1,249400,Vente,RUE,D ORADOUR SUR GLANE,75015,ISSY-LES-MOULINEAUX,92,368.0,32.88,1,Appartement,34.0,1.0,2.282171,48.831289
2,249800,Vente,RUE,CHATEAUBRIAND,75008,PARIS 08,75,511.0,33.66,1,Appartement,34.0,2.0,2.301586,48.873205
3,249943,Vente,RUE,DE TURENNE,75003,PARIS 03,75,7.0,50.67,1,Appartement,59.0,2.0,2.364598,48.857983
4,250150,Vente,RUE,DU PONT AUX CHOUX,75003,PARIS 03,75,25.0,19.70,1,Appartement,20.0,1.0,2.365183,48.861299


In [20]:
dvf_test.to_csv("./data_test_clean.csv")